In [67]:
#imports
import os
import shutil
import subprocess

In [68]:
DATASET_1 = 'Data_1'
DATASET_2 = 'Data_2'
DATASETS = [DATASET_1]

SCENARIO_1 = 'Ulds_scenario_1'
SCENARIO_2a = 'Ulds_scenario_2a'
SCENARIO_2b = 'Ulds_scenario_2b'
SCENARIOS = [SCENARIO_1, SCENARIO_2a, SCENARIO_2b]

ITEMS_BELOW_LEVELS = [20]

PATH = './Data'

In [69]:
def createOutputFolderIfNotExists(path):
    isExist = os.path.exists(path)
    if isExist:
        shutil.rmtree(path)
        os.makedirs(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)
    print("The new directory is created!")

In [70]:
def copyFile(file, destination):
    shutil.copy(file, destination)

In [71]:
def copyFilesInFolder(source_folder, destination_folder):
    # fetch all files
    for file_name in os.listdir(source_folder):
        # construct full file path
        source = source_folder + file_name
        destination = destination_folder + file_name
        # copy only files
        if os.path.isfile(source):
            copyFile(source, destination)
            print('copied', file_name)    

In [72]:
import json

def readJson(filePathImport):
    #Read file in
    with open(filePathImport) as json_file:
        json_file = json.load(json_file)
        return json_file

In [73]:
def writeJson(filePathExport, jsonFile):
    with open(filePathExport, 'w', encoding='utf-8') as f:
        json.dump(jsonFile, f, ensure_ascii=False, indent=4)

In [74]:
def makeFolderStructure(path_to_benchmark_scenario):
    createOutputFolderIfNotExists(path_to_benchmark_scenario + "/todo")
    for level in ITEMS_BELOW_LEVELS:
        createOutputFolderIfNotExists(path_to_benchmark_scenario + "/selected/" + str(level))
        createOutputFolderIfNotExists(path_to_benchmark_scenario + "/selected/" + str(level) + "_clipped")

In [75]:
def executeJavaTool(itemsbelow, input_path, output_path):
    subprocess.run(['java', '-jar', 'Select-Files.jar',
                    '--itemsbelow', str(itemsbelow), input_path, output_path], capture_output=True)

In [77]:
def makeArchive(path_to_selected, level, zipped_file_name):
    subprocess.run(['tar', '-czvf', zipped_file_name  + ".tgz", "-C", 
                    path_to_selected + str(level), '.', '--exclude=*.DS_Store'], capture_output=True)
    subprocess.run(['tar', '-czvf', zipped_file_name + "_clipped"  + ".tgz", "-C", 
                path_to_selected + str(level) + "_clipped", '.', '--exclude=*.DS_Store'], capture_output=True)

In [78]:
def prepare(dataset, scenario, path_to_dataset):
    path_to_benchmark_scenario = path_to_dataset + "/4_Benchmark" + "/" + scenario 
    path_to_jobs_full = path_to_dataset + "/2_AeJobs" + "/" + scenario + "/"
    path_to_todo = path_to_benchmark_scenario + "/todo/"
    path_to_selected = path_to_benchmark_scenario + "/selected/"
    
    makeFolderStructure(path_to_benchmark_scenario)
    
    # copy AeJobs to /todo
    copyFilesInFolder(path_to_jobs_full, path_to_todo)
    
    for level in ITEMS_BELOW_LEVELS:
        zipped_file_name = path_to_selected + dataset + "_" + scenario + "_" + str(level)
        
        # execute java tool to seperate ULDs based on number of placed items 
        executeJavaTool(level, path_to_todo, path_to_selected + str(level))
        
        # make archive out of folders in /selected
        makeArchive(path_to_selected, level, zipped_file_name)

In [79]:
for dataset in DATASETS:
    for scenario in SCENARIOS:
        path_to_dataset = PATH + "/" + dataset
        prepare(dataset, scenario, path_to_dataset)

The new directory is created!
The new directory is created!
The new directory is created!
copied Data_1_Ulds_scenario_1_LH8222-25NOV15-FRA-MEX_pmc_md11f_md_cad-7.json
copied Data_1_Ulds_scenario_1_LH8368-25NOV15-FRA-BOM_pmc_md11f_md-18.json
copied Data_1_Ulds_scenario_1_LH8340-29NOV15-FRA-IST_pge_md11f_md-7.json
copied Data_1_Ulds_scenario_1_LH8400-29NOV15-FRA-PVG_pmc_md11f_md-9.json
copied Data_1_Ulds_scenario_1_LH8088-29NOV15-FRA-LEJ_pmc_md11f_md-2.json
copied Data_1_Ulds_scenario_1_LH8048-28NOV15-FRA-LAX_pmc_md11f_md-7.json
copied Data_1_Ulds_scenario_1_LH8452-25NOV15-FRA-DMM_pmc_F_ld-5.json
copied Data_1_Ulds_scenario_1_LH8202-25NOV15-FRA-ORD_pmc_md11f_md-0.json
copied Data_1_Ulds_scenario_1_LH8202-25NOV15-FRA-DFW_pmc_md11f_md-4.json
copied Data_1_Ulds_scenario_1_LH8414-25NOV15-FRA-ALA_pmc_md11f_md-0.json
copied Data_1_Ulds_scenario_1_LH8414-25NOV15-FRA-CAN_pmc_md11f_md-3.json
copied Data_1_Ulds_scenario_1_LH8340-24NOV15-FRA-TLV_pmc_md11f_md-2.json
copied Data_1_Ulds_scenario_1_LH8